# Night Log

In [ ]:
# Parameters. Set defaults here.
# Times Square replaces this cell with the user's parameters.
record_limit = '999'
day_obs = 'TODAY'  # TODAY, YESTERDAY, YYYY-MM-DD
number_of_days = '1'  # Total number of days of data to display (ending on day_obs)

In [ ]:
# Only use packages available in the Rubin Science Platform
import requests
from collections import defaultdict
import pandas as pd
from pprint import pp
from urllib.parse import urlencode
from IPython.display import display, Markdown, display_markdown
from matplotlib import pyplot as plt
import os
from datetime import datetime, date, timedelta

In [ ]:
limit = int(record_limit)

match day_obs.lower():
    case 'today':
        date = datetime.now().date()
    case 'yesterday':
        date = datetime.now().date()-timedelta(days=1)
    case _:
        date = datetime.strptime(dd, '%Y-%m-%d').date()

days = int(number_of_days)

# Thus: [min_day_obs,max_day_obs)
min_day_obs = (date - timedelta(days=days-1)).strftime('%Y%m%d') # Inclusive
max_day_obs = (date + timedelta(days=1)).strftime('%Y%m%d') # prep for Exclusive

response_timeout = 3.05  # seconds, how long to wait for connection
read_timeout = 20  # seconds
timeout = (float(response_timeout), float(read_timeout))

summit = 'https://summit-lsp.lsst.codes'
usdf = 'https://usdf-rsp-dev.slac.stanford.edu'
tucson = 'https://tucson-teststand.lsst.codes'

# Use server=tucson for dev testing
server = os.environ.get('EXTERNAL_INSTANCE_URL', summit)


In [ ]:
print(f'Report from {server} over {number_of_days} nights'
      f' from {min_day_obs} to {date}. ')

In [ ]:
# For Times Square, comment out next line and past next cell with contents of local python file.
from lsst.ts.logging_and_reporting.source_adapters import ExposurelogAdapter, NarrativelogAdapter
# Once our logrep package has been installed in RSP, we can use the simpler "import"

In [ ]:
# Paste contents of source_adapters.py in new cell below this one.

# Exposure Log

In [ ]:
service_adapter = ExposurelogAdapter(server_url=server)
source = f'{server}'
try:
    recs = service_adapter.get_messages(limit=limit,
                                        min_day_obs=min_day_obs,
                                        max_day_obs=max_day_obs,
                                       )
except Exception as err:
    recs = []
    msg = f'ERROR getting records from {server=}: {err=}'
    raise Exception(msg)

In [ ]:
print(f'Retrieved {len(recs)} records')

In [ ]:
if recs:
    ignore_fields = ['id']
    new_column_names = dict(message_text='message',
                            date_added='date'
                         )
    df = pd.DataFrame(recs).rename(columns=new_column_names)
    user_df = df[['message', 'date']]
    
    display_markdown(f'### Exposure log for {number_of_days} days {min_day_obs} to {max_day_obs}', raw=True)
    display(user_df)

In [ ]:
gaps = service_adapter.get_observation_gaps()
for instrument, day_gaps in gaps.items():
    display_markdown(f'### Date vs Observation Gap (minutes) for {instrument=!s}', raw=True)
    x,y = zip(*day_gaps.items())
    df = pd.DataFrame(dict(day=x,minutes=y))
    df.plot.bar(x='day', y='minutes', title=f'{instrument=!s}')

# Narrative Log

In [ ]:
service_adapter = NarrativelogAdapter(server_url=server)
try:
    # date like '2000-01-02 12:00:00'
    # str(datetime(2000, 1, 2, 12, 0, 0))
    recs = service_adapter.get_messages(limit=limit,
                                        min_date_end=str(datetime.strptime(min_day_obs,'%Y%m%d')),
                                        max_date_end=str(datetime.strptime(max_day_obs,'%Y%m%d')),
                                       )
except Exception as err:
    recs = []
    msg = f'ERROR getting records from {server=}: {err=}'
    raise Exception(msg)

print(f'Retrieved {len(recs)} records.')

In [ ]:
ignore_fields = ['id']
new_column_names = dict(message_text='message',
                        date_added='date'
                     )
if recs:
    df = pd.DataFrame(recs).rename(columns=new_column_names)
    user_df = df[['message', 'date']]

    display(Markdown(f'## Narrative log (format A) for {number_of_days} days {min_day_obs} to {max_day_obs}'))
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        display(user_df)
        
    display(Markdown(f'## Narrative log (format B)'))
    for index,row in user_df.iterrows():
        print(f"{datetime.fromisoformat(user_df.iloc[0]['date']).date()}: "
              f"{row['message']}"
        )